### This Jupyter notebook will be used to determine the user defines variables

#### Step 1 - Retrieve Location of Desired Study Area
Using either a shapefile, coordinate, or WKT the study area for where satalite imagery will be downloaded is to be determined

In [12]:
#import requirements
import pandas as pd
import geopandas as gpd
import asf_search as asf

To begin, we will need to craft a suitable WKT. ASF's [Vertex](https://search.asf.alaska.edu) can be helpful in this regard, as it allows you to draw on a map, or import a geospatial file such as a shapefile or geojson, after which a WKT string can be copied and used elsewhere.

In [11]:
#select location using a WKT obtained from ASF's Vertex search site or copied from a geodatabase (must resemble: 'POLYGON((-85.5523 64.8284,-84.9823 64.8284,-84.9823 65.3214,-85.5523 65.3214,-85.5523 64.8284))')

location = 'POLYGON((-85.5523 64.8284,-84.9823 64.8284,-84.9823 65.3214,-85.5523 65.3214,-85.5523 64.8284))'

In [ ]:
#Here 
compAmount = 2
compCount = 1

geo_results1 = asf.geo_search(
        intersectsWith=location,
        platform=asf.PLATFORM.SENTINEL1,
        start='2010-01-01',
        end='2010-02-01')